In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from census import Census

In [2]:
# Census API key
from courtCensusKey import censusKey

In [3]:
complete_url='https://api.census.gov/data/timeseries/intltrade/exports/hs'+\
            '?get=DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC'+\
            '&E_COMMODITY=3926201010'+\
            '&YEAR=2019'+\
            '&MONTH=*'

In [4]:
response=requests.get(complete_url)
response

<Response [200]>

In [5]:
pd.DataFrame(response.json())

,0,1,2,3,4,5,6,7
0,DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC,E_COMMODITY,YEAR,MONTH
1,-,TOTAL FOR ALL DISTRICTS,1109949,1109949,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,01
2,07,"OGDENSBURG, NY",99903,99903,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,01
3,09,"BUFFALO, NY",237669,237669,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,01
4,11,"PHILADELPHIA, PA",2852,2852,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,01
...,...,...,...,...,...,...,...,...
330,49,"SAN JUAN, PR",72819,519368,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,12
331,52,"MIAMI, FL",67122,1557770,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,12
332,53,"HOUSTON-GALVESTON, TX",0,39082,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,12
333,55,"DALLAS-FORT WORTH, TX",0,82464,"GLOVES,SEAMLESS,SURGICAL & MEDICAL,OF PLASTIC",3926201010,2019,12


In [6]:
#Setting variables 
year='2019'
month='*'
cols='DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC'


In [ ]:
#Base URL 
base_url = 'https://api.census.gov/data/timeseries/intltrade/exports/hs'

complete_url = f'{base_url}?get={cols}&YEAR={year}&MONTH={month}'

# HTS Codes and what they represent
# 0901 - Coffee
# 480300 - Toilet Paper
# 2203 - Beer
# 2204 - Wine
# Hard alcohol - 2207

# Items in US top Exports
# Medical Equipment - $83 billion
# Plastics - $60 billion
# Pharmaceuticals - $47 billion
    # 3002 - Vaccines, Blood, and Immunological products
    # 3003 - Medication (unmeasured doses)
    # 3004 - Medication (measured doses)
    # 3005 - Medically-treated Bandages
    # 3006 - Surgical Equipment
# Organic chemicals - $39 billion

hts_list=['3002','3003','3004','3005','3006']
df_list=[]

for each_hts in hts_list:
    hts_code=each_hts
    query_params={
        'get': 'CTY_CODE,CTY_NAME,DF,DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC',
        'E_COMMODITY': hts_code,
        'YEAR': '2019',
        'MONTH': "*"
    }
    max_response=requests.get(base_url, params=query_params)
    #print(max_response.json())
    df_list.append(max_response.json()[1:])
#print(df_list)

In [ ]:
productList = []
for product in df_list:
    productList += product

In [ ]:
exportData = pd.DataFrame(productList, columns=max_response.json()[0])
exportData

In [ ]:
exportData['E_COMMODITY_SDESC'].unique()

In [ ]:
# Clean up column names
exportData = exportData.rename(columns={"CTY_CODE": "COUNTRY_CODE", "CTY_NAME": "COUNTRY_NAME", "DF": "DOMESTIC_FOREIGN_CODE", "DIST_NAME": "DISTRICT_NAME", "ALL_VAL_MO": "ALL_VALUES_MONTH", "ALL_VAL_YR": "ALL_VALUES_YEAR", "E_COMMODITY_SDESC": "COMMODITY_DESCRIPTION", "E_COMMODITY": "COMMODITY"})

In [ ]:
# Clean up commodity names
exportData.loc[(exportData.COMMODITY_DESCRIPTION == 'HUMAN BLOOD; ANIMAL BLOO'), 'COMMODITY_DESCRIPTION'] = 'Vaccines, Blood, and Immunological products'
exportData.loc[(exportData.COMMODITY_DESCRIPTION == 'MEDICAMENTS NESOI OF MIXTU'), 'COMMODITY_DESCRIPTION'] = 'Medication (unmeasured doses)'
exportData.loc[(exportData.COMMODITY_DESCRIPTION == 'MEDICAMENTS NESOI, MIXED O'), 'COMMODITY_DESCRIPTION'] = 'Medication (measured doses)'
exportData.loc[(exportData.COMMODITY_DESCRIPTION == 'BANDAGES ETC COATED ETC OR'), 'COMMODITY_DESCRIPTION'] = 'Medically-Treated Bandages'
exportData.loc[(exportData.COMMODITY_DESCRIPTION == 'PHARMACEUTICAL GOODS IN NOTE 4 TO CHAPTER 30'), 'COMMODITY_DESCRIPTION'] = 'Surgical Equipment'
exportData

In [ ]:
# Create table with only totals for visualization
totalExportData = exportData[(exportData['COUNTRY_NAME'] == "TOTAL FOR ALL COUNTRIES") & (exportData['DISTRICT_NAME'] == "TOTAL FOR ALL DISTRICTS")]
# Create charts with removed totals so they won't skew numbers
exportData = exportData[exportData['COUNTRY_NAME'] != "TOTAL FOR ALL COUNTRIES"]
exportData = exportData[exportData['DISTRICT_NAME'] != "TOTAL FOR ALL DISTRICTS"]
exportData

In [ ]:
# Save as a csv
# Using encoding="utf-8" to avoid issues later
exportData.to_csv("Resources/exportData2019.csv", encoding="utf-8", index=False)
totalExportData.to_csv("Resources/totalExportData2019.csv", encoding="utf-8", index=False)